# PumpController Setup

Now we can start mixing some colors in the real world. Similar to before, we import the necessary functions. This time we have two extra functions helping us to connect to the **PumpController**

In [ ]:
from pump_controller import PumpController, get_serial_port, list_serial_ports
from pump_controller import visualize_rgb
import numpy as np

You can read more about the different classes and methods in the pump_controller module [here](https://www.student.dtu.dk/~s193903/47332/).

Now we initialize the pumpbot. We need to figure out which port that the controller is connected to your computer. The `get_serial_port` should automatically do this for you, but things have a way of failing when you need them not to. That is why you can call the `list_serial_ports` function to see all the ports on your computer, and you can simply use the correct port as a string input instead of using the `get_serial_port` function. The `cell_volume` and `drain_time` properties are already defined to 20 mL and 20 seconds, respectively, but you have the option of changing them here. Notice that a folder called *logs* is created with a file with the current timestamp in it - this is where the colors that you mix on this controller in this session will be stored.


You might want to change the `config_file` to your config file

When initializing the bot, if the "Arduino is ready" command doesn't show up immediately, press the reset button on the Arduino.

In [ ]:
pumpbot = PumpController(ser_port = get_serial_port(), cell_volume = 20, drain_time = 20, config_file = 'config_files/config.json')

# list_serial_ports()
# pumpbot = PumpController(ser_port = '/dev/ttyUSB0', cell_volume = 20, drain_time = 20)

Before doing anything else, we should purge all the pumps. This means filling all of the tubes with their respective liquids. We do this by using the `purge_pump` function, which takes the pump name ('R', 'G', 'B', 'Y', 'W', 'D') and the time to run the pump, as variables. Do this one-by-one until all tubes are filled with liquid. The drain tubes of course do not need to be purged. You can then drain the test cell using the `drain` function which by itself drains the whole cell for the defined drain time in the previous cell, but you can also define a custom drain time here. If needed you can also flush the cell with water using the `flush` function and then drain again.

In [ ]:
pumpbot.purge_pump('R', 3)

# pumpbot.purge_pump('G', 3)
# pumpbot.purge_pump('B', 3)
# pumpbot.purge_pump('Y', 3)
# pumpbot.purge_pump('W', 3)

# Drain:
# pumpbot.drain()

# Drain for custom time:
# pumpbot.drain(drain_time = 10)

# Flush:
# pumpbot.flush()

At this point, your teacher would give you a color that you have to match, where you do not know anything about the mixture. You can pour this color into the test cell using a syringe and the extra hole in the lid of the test cell. Use the `measure` function to measure this color and store this in the `target_color` property of the pumpbot. 


If at any point you feel like making your own target color and trying to match it, you can do it in the same way as with the **SilicoPumpController** and the `change_target` function

In [ ]:
pumpbot.target_color = pumpbot.measure()
print(pumpbot.target_color)

# Make your own target color:
# pumpbot.change_target([0.1, 0.2, 0.3, 0.4])
# print(pumpbot.target_mixture)
# print(pumpbot.target_color)

After the target color is measured, you should empty the test cell. This can be done using the `reset` function. This function drains the cell, flushes it with water and then drains the cell again. This is equivalent to calling the `drain`, `flush` and `drain` functions.

In [ ]:
pumpbot.reset()

# Equivalent to:
# pumpbot.drain()
# pumpbot.flush()
# pumpbot.drain()


We again define the `color_difference` function to find the match score between two colors

In [ ]:
# Difference between mixed and target colors:
def color_difference(mixed_color, target_color):

    mixed_color = np.array(mixed_color)
    target_color = np.array(target_color)
    # Calculate the sum of root mean squared differences between mixed color and target color
    rmse = np.sqrt(np.mean((mixed_color - target_color)**2, axis=-1))
    return np.sum(rmse)

Now you can mix a color with a mixture that you decide, and calculate the match score with the target color

In [ ]:
mixture_to_mix = [0.4, 0.3, 0.2, 0.1]
measured_color = pumpbot.mix_color(mixture_to_mix)

print(f"Measured Color: {measured_color}")
print(f"Target Color: {pumpbot.target_color}")

In [ ]:
score = color_difference(measured_color, pumpbot.target_color)
print(f"Score: {score}")

Again, you can visualize this newly mixed color

In [ ]:
visualize_rgb(mixture = mixture_to_mix,
              rgb = measured_color,
              pump_controller = pumpbot,
              target = pumpbot.target_color,
              score = score)

This concludes the **PumpController** introduction. Now you can start mixing your own colors and trying to match the colors manually. You might find that this isn't the most optimal way - an algorithm to decide these colors might be a better way!